In [1]:
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.time import Time
import datetime

def df_to_X_y(df, window_size=30):
    df_as_np = df.to_numpy()
    X,y = [], []
    for i in range(len(df_as_np)-window_size):
        if abs(df_as_np[i+window_size][0] - (window_size*(df_as_np[i-1][0]-df_as_np[i][0])+df_as_np[i][0])) < 1:
            X.append(df_as_np[i:i+window_size])
            y.append(df_as_np[i+window_size][-1])
    return np.array(X), np.array(y)

In [2]:
df = pd.read_csv("only_GRBAlpha_logxyz_firm3.csv")
df

,time,x,y,z,log_cps
0,2022-08-08 23:45:23.405963008,198.004559,-264.929213,-455.839536,1.981139
1,2022-08-08 23:45:27.405979904,226.530494,-367.147747,-361.979136,1.985651
2,2022-08-08 23:45:31.405984000,226.612597,-453.022604,-246.006367,1.984527
3,2022-08-08 23:45:35.405987072,202.300037,-512.695303,-115.236975,2.024280
4,2022-08-08 23:45:39.405988096,160.243127,-539.293648,22.453799,1.973128
...,...,...,...,...,...
1977610,2023-03-31 22:50:50.878717952,292.031434,25.065833,-446.488696,3.933892
1977611,2023-03-31 22:50:51.878717952,264.282458,17.898810,-463.809937,3.940467
1977612,2023-03-31 22:50:52.878717952,235.360313,11.688833,-479.350154,3.935809
1977613,2023-03-31 22:50:53.878717952,204.902403,6.686862,-493.253822,3.949341


In [3]:
grb = ["2022-08-15 14:34:56.390",
"2022-08-16 00:08:32.360",
"2022-08-26 10:21:19.720",
"2022-08-29 14:37:50.308",
"2022-09-04 12:13:25.000",
"2022-09-15 05:13:40.900",
"2022-09-26 10:38:19.750",
"2022-09-27 05:36:20.950",
"2022-10-09 13:16:59.99",
"2022-10-14 07:12:28.77",
"2022-10-14 17:27:39.767",
"2022-10-20 05:23:52.830",
"2022-10-22 22:55:48.91",
"2022-10-29 01:05:27.310",
"2022-11-07 01:22:52.46",
"2022-11-11 05:54:27.320",
"2022-11-11 11:39:28.529",
"2022-11-11 13:49:11.120",
"2022-11-12 06:18:04.36",
"2022-11-19 15:02:53.640",
"2022-11-22 02:43:26.65",
"2022-11-27 08:27:10.520",
"2022-12-06 12:22:47.360",
"2022-12-29 18:18:17.662",
"2023-01-02 00:22:12.970",
"2023-02-04 21:44:27.200",
"2023-02-07 04:40:45.260",
"2023-02-10 03:00:26.000",
"2023-02-11 15:45:01.041",
"2023-02-11 17:36:11.870",
"2023-02-22 13:46:10.615",
"2023-03-04 14:35:49.140",
"2023-03-05 13:40:29.28",
"2023-03-07 15:44:06.67",
"2023-03-08 22:41:39.690",
"2023-03-12 06:07:09.590"]

# converts burst time into julian date - int of first data
grb_list = []
for i in range(len(grb)):
    t = Time(grb[i], format='iso', scale='utc')
    grb_list.append(t.jd - 2459319)
grb_list

[488.10759710660204,
 488.5059300926514,
 498.9314782405272,
 502.10961004626006,
 508.0093171298504,
 518.717834490817,
 529.9432841436937,
 530.7335758102126,
 543.053472106345,
 547.800332986284,
 548.2275435994379,
 553.7249170136638,
 556.4554271991365,
 562.5454549770802,
 571.5575516205281,
 575.7461495371535,
 575.9857468632981,
 576.0758231482469,
 576.7625504629686,
 584.1270097224042,
 586.6135028935969,
 591.852205092553,
 601.0158259258606,
 624.2627044213004,
 627.5154278934933,
 661.4058703705668,
 663.6949682869017,
 666.6253009261563,
 668.1562620485201,
 668.233470717445,
 679.0737339700572,
 689.1082076388411,
 690.0697833332233,
 692.1556327547878,
 693.4455982637592,
 696.7549721063115]

In [4]:
# converts time into julian date - int of first data

new_time = []
for i in range(len(df["time"])):
    thyme = Time(df["time"][i],format='iso', scale='utc')
    new_time.append(thyme.jd - 2459319)
df["time"] = new_time

In [ ]:
# makes array of known grbs and removes them from data

grb_dataq = []
for value in grb_list:
    i = np.argmin(abs(value - df["time"]))
    start = max(0, i - 256)
    end = min(len(df["time"]), i + 32)
    grb_dataq.append(df[start:end].to_numpy())
    df.drop(df.index[start:end],0,inplace=True)


In [ ]:
n_grb = len(grb_dataq)
fig, axs = plt.subplots(n_grb,4,figsize=(20,4*n_grb))

for i in range(n_grb):
    
    axs[i,0].plot(grb_dataq[i][:,0],grb_dataq[i][:,-1],label="{}".format(i))
    axs[i,0].legend()
    axs[i,1].plot(grb_dataq[i][-64:,0],grb_dataq[i][-64:,-1])
    
    axs[i,2].plot(grb_dataq[i][:,0],10**(grb_dataq[i][:,-1]),label="{}".format(i))
    axs[i,3].plot(grb_dataq[i][-64:,0],10**(grb_dataq[i][-64:,-1]))

In [ ]:
stop here

In [ ]:
plt.plot(grb_dataq[5][:,-1])

In [ ]:
np.shape(grb_dataq)

In [ ]:
del grb_dataq[5]

In [ ]:
np.shape(grb_dataq)

In [ ]:
for GRB in grb_dataq:
    GRB_to_dic = pd.DataFrame(GRB)
    GRB_to_dic.to_csv("Data/OnlyGRBs-alpha/{}.csv".format(int((GRB_to_dic[0][0])*10)))

In [ ]:
grb_dataq[0][:,1:5]

In [ ]:
df.to_csv('Data/GRBAlpha_without_GRBs.csv')